This code first reads in a CSV file that is produced from the notebook "Simulation CSV". 
The objects is to analyze which craters are detroyed due to being covered by another crater, and produce a CSV file of just the surviving craters. 

For each crater j, the program calculates the rim loss due to all of the other craters.
It does this by:
First, filtering the craters formed after crater j, bcause only craters formed after can destroy a crater.
Then for each of those craters, it filters craters that overlap with j, because only craters that overlap with j will contribute towards erasure.
Next, it takes the filtered craters, and calculates the rim loss on crater j due to each crater.
Then it adds up the rim loss due to each crater.
If this is greater than 75% of the rim, the crater is cosndiered destroyed, and removed from the CSV list.

In [1]:
# Imports
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.optimize import minimize
from tqdm import tqdm
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER
from matplotlib.patches import Circle
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed

In [3]:
MOON_RADIUS = 1737.1
#MOON_CIRCUMFERENCE = 10_917

In [2]:
# Functions 
"""
Original Functions: Angular Seperation, Check overlap, and Calculate Rim Loss
"""

# Angular separation using Vincenty formula
"""
Calculates the angular seperation between two craters, using the vincenty formula
Args:
    lon1: Longitude of crater 1
    lat1: Latitude of crater 1
    lon2: Longitude of crater 2
    lat2: Latitude of crater 2  
Returns: Angular seperation between two craters
"""
def angular_sep(lon1, lat1, lon2, lat2):
    # Convert degrees to radians
    lon1, lat1, lon2, lat2 = np.radians([lon1, lat1, lon2, lat2])
    delta_lon = lon2 - lon1
    numerator = np.sqrt((np.cos(lat2) * np.sin(delta_lon))**2 + (np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(delta_lon))**2)
    denominator = np.sin(lat1) * np.sin(lat2) + np.cos(lat1) * np.cos(lat2) * np.cos(delta_lon)
    return np.arctan2(numerator, denominator)

# Check overlap between crater 1 and 2
"""
Checks if crater 1 and crater 2 overlap with each other
Args:
     crater 1: first crater in comparison
     crater 2: second crater in comparison
Returns: Boolean value: True if craters overlap, false if craters do not overlap
"""
def check_overlap(crater1, crater2):
    lon1, lat1, size1 = crater1[0], crater1[1], crater1[2]
    lon2, lat2, size2 = crater2[0], crater2[1], crater2[2]
    separation = angular_sep(lon1, lat1, lon2, lat2)
    size1_rad = size1 / MOON_RADIUS
    size2_rad = size2 / MOON_RADIUS
    return separation < (size1_rad + size2_rad) / 2

# Calculate rim loss on crater 1 due to crater 2
"""
Calculates the rim loss on crater 1 due to crater 2, as an estimate of the % of area lost of crater 1, in order to determine if it is erased.
Args:
     crater 1: first crater in comparison
     crater 2: second crater in comparison
Returns: Rim loss on crater 1 due to crater 2
"""
def calculate_rim_loss_on_crater1(crater1, crater2):
    lon1, lat1, diam1 = crater1
    lon2, lat2, diam2 = crater2
    # Convert diameters to radii
    r1 = diam1 / 2
    r2 = diam2 / 2
    # Convert degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    # Calculate the central angle between the crater centers using spherical law of cosines
    delta_sigma = np.arccos(np.sin(lat1) * np.sin(lat2) + np.cos(lat1) * np.cos(lat2) * np.cos(lon2 - lon1))
    d = MOON_RADIUS * delta_sigma  # Distance between crater centers
    """
    x is the distance from the center of crater 1 to the nearest point of intersection along the line connecting 
    crater 1 and 2.
    z is the cosine of the angle that we want to find.
    """
    # Check for overlap
    if d < r1 + r2:
        # Calculate the angle subtended by the arc of Crater 1's rim that is intercepted
        # Using the formula for intersecting circles on a sphere
        x = (d**2 + r1**2 - r2**2) / (2 * d)
        z = x / r1
        z = np.clip(z, -1.0, 1.0)  # Ensuring the value is within the domain of arccos
        a1 = np.arccos(z) * 2
        # Calculate rim loss for Crater 1 as a percentage of its total circumference
        rim_loss = (a1 / (2 * np.pi)) * 100

        return rim_loss
    else:
        return 0

In [5]:
# Surviving craters function 
"""
Define the csv path used for crater data before erasure. 
Calculates the craters that survive erasure
Args:
     csv path: Define the path used to feed in simulation data in order to calculate erasure
     threshold loss: % of rim loss required to be considered erased, and removed from CSV 
Returns: CSV file downloaded to computer of craters that survive erasure
"""

def get_surviving_craters(csv_path, threshold_loss=0.75):
    # Read CSV file into DataFrame
    craters_df = pd.read_csv(csv_path)
    # Convert radians to degrees
    craters_df['Longitude'] = np.degrees(craters_df['Longitude']) % 360
    craters_df['Latitude'] = np.degrees(craters_df['Latitude'])
    # Convert dataframe to tuples for faster processing
    craters_list = craters_df.to_records(index=False).tolist()
    surviving_craters = []

    for j in tqdm(range(len(craters_list)), desc="Processing Craters"):
        crater_j = craters_list[j]
        total_rim_loss = 0
        # Filter for craters that overlap with crater j and formed after crater j + checks
        S_all_after = [crater_i for crater_i in craters_list 
                       if crater_i[3] > crater_j[3]  # Check formation time
                       # Check that seperation in Lon and Lat is greater than radii combined
                       #and np.radians(min(abs(crater_i[0] - crater_j[0]), abs(crater_i[0] - crater_j[0] - 360), (360 - abs(crater_i[0] - crater_j[0])))) <= (crater_j[2] / MOON_RADIUS + crater_i[2] / MOON_RADIUS)
                       and np.radians(abs(crater_i[1] - crater_j[1])) <= (crater_j[2] / MOON_RADIUS + crater_i[2] / MOON_RADIUS) 
                       and check_overlap(crater_j[:3], crater_i[:3])]

        for crater_i in S_all_after:
            rim_loss = calculate_rim_loss_on_crater1(crater_j[:3], crater_i[:3])
            total_rim_loss += rim_loss

        # Check if crater survives
        if total_rim_loss <= threshold_loss:
            surviving_craters.append(crater_j)

    # Convert surviving craters back to DataFrame
    surviving_craters_df = pd.DataFrame(surviving_craters, columns=craters_df.columns)

    # Define the output directory and file name (timestamped)
    output_dir = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Surviving craters CSV files"
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f'surviving_craters_{timestamp}.csv'
    output_path = os.path.join(output_dir, output_filename)

    # Save surviving craters to a CSV file
    surviving_craters_df.to_csv(output_path, index=False)

    return surviving_craters_df

In [6]:
# Calculate surviving craters for dataset with 1000 craters
csv_path = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_1000.csv"
surviving_craters_df = get_surviving_craters(csv_path)

Processing Craters: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 981/981 [00:03<00:00, 326.94it/s]


In [ ]:
# Track speed for 100,000 craters
csv_path2 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_100k.csv"
surviving_craters_df = get_surviving_craters(csv_path2)

Processing Craters:  26%|█████████████▍                                     | 26563/100598 [1:25:13<2:35:17,  7.95it/s]

In [7]:
# Track speed for 20,000 craters
csv_path3 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_20k.csv"
surviving_craters_df = get_surviving_craters(csv_path3)

Processing Craters: 100%|████████████████████████████████████████████████████████| 20372/20372 [07:11<00:00, 47.27it/s]


In [30]:
# Track speed for 50,000 craters
csv_path20 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_50k.csv"
surviving_craters_df = get_surviving_craters(csv_path20)

Processing Craters: 100%|████████████████████████████████████████████████████████| 50534/50534 [50:29<00:00, 16.68it/s]


In [31]:
# Track speed for 70,000 craters
csv_path20 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_70k.csv"
surviving_craters_df = get_surviving_craters(csv_path20)

# 50% 1 hour 20 mins
# 66% 1 hour 31 min 
# 88% 1 hour 40 mins
# 100% 1 hour 41 mins

# Final 50% takes about 1/4 of the time it takes to get to 1/2

Processing Craters: 100%|██████████████████████████████████████████████████████| 70800/70800 [1:41:24<00:00, 11.64it/s]


In [32]:
# Track speed for 30,000 craters
csv_path20 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_30k.csv"
surviving_craters_df = get_surviving_craters(csv_path20)

Processing Craters: 100%|████████████████████████████████████████████████████████| 30336/30336 [15:38<00:00, 32.33it/s]


In [36]:
# Track speed for 40,000 craters
csv_path21 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_40kk.csv"
surviving_craters_df = get_surviving_craters(csv_path21)

Processing Craters: 100%|████████████████████████████████████████████████████████| 40668/40668 [29:57<00:00, 22.63it/s]


Changing the number of steps: 1, 3, 10, 30, 100
Holding constant # of craters: 20,000
                      dmin   : 0.27

In [15]:
# Track speed for 1 time step
csv_path4 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_1step_10k.csv"
surviving_craters_df = get_surviving_craters(csv_path4)

Processing Craters: 100%|███████████████████████████████████████████████████████| 10069/10069 [00:18<00:00, 532.52it/s]


In [17]:
# Track speed for 3 time step
csv_path5 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_3step_10k.csv"
surviving_craters_df = get_surviving_craters(csv_path5)

Processing Craters: 100%|████████████████████████████████████████████████████████| 10154/10154 [03:32<00:00, 47.82it/s]


In [18]:
# Track speed for 10 time step
csv_path6 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_10step_10k.csv"
surviving_craters_df = get_surviving_craters(csv_path6)

Processing Craters: 100%|████████████████████████████████████████████████████████| 10187/10187 [04:55<00:00, 34.47it/s]


In [20]:
# Track speed for 30 time step
csv_path7 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_30step_10k.csv"
surviving_craters_df = get_surviving_craters(csv_path7)

Processing Craters: 100%|████████████████████████████████████████████████████████| 10033/10033 [04:53<00:00, 34.15it/s]


In [22]:
# Track speed for 100 time step
csv_path8 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_100step_10k.csv"
surviving_craters_df = get_surviving_craters(csv_path8)

Processing Craters: 100%|████████████████████████████████████████████████████████| 10200/10200 [05:21<00:00, 31.74it/s]


Changing d min (km): 100, 30, 10, 3, 1
Holding constant: # steps: 10
number of craters: 10,000

In [23]:
# Track speed for dmin = 100
csv_path9 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_dmin_100_10k.csv"
surviving_craters_df = get_surviving_craters(csv_path9)

Processing Craters: 100%|████████████████████████████████████████████████████████| 10001/10001 [29:18<00:00,  5.69it/s]


In [25]:
# Track speed for dmin = 30
csv_path10 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_dmin_30_10k.csv"
surviving_craters_df = get_surviving_craters(csv_path10)

Processing Craters: 100%|████████████████████████████████████████████████████████| 10009/10009 [11:15<00:00, 14.82it/s]


In [26]:
# Track speed for dmin = 10
csv_path11 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_dmin_10_10k.csv"
surviving_craters_df = get_surviving_craters(csv_path11)

Processing Craters: 100%|████████████████████████████████████████████████████████| 10115/10115 [06:45<00:00, 24.98it/s]


In [27]:
# Track speed for dmin = 3
csv_path12 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_dmin_3_10k.csv"
surviving_craters_df = get_surviving_craters(csv_path12)

Processing Craters: 100%|████████████████████████████████████████████████████████| 10001/10001 [03:58<00:00, 41.95it/s]


In [28]:
# Track speed for dmin = 1
csv_path13 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_dmin_1_10k.csv"
surviving_craters_df = get_surviving_craters(csv_path13)

Processing Craters: 100%|██████████████████████████████████████████████████████████| 9988/9988 [02:51<00:00, 58.36it/s]


In [29]:
# Track speed for dmin = 0.27
csv_path14 = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\crater_dmin_0.27_10k.csv"
surviving_craters_df = get_surviving_craters(csv_path14)

Processing Craters: 100%|████████████████████████████████████████████████████████| 10011/10011 [02:09<00:00, 77.02it/s]


In [5]:
# 1k craters, 27s
csv_path = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\craters_1k.csv"
surviving_craters_df = get_surviving_craters(csv_path)

Processing Craters: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 1016/1016 [00:01<00:00, 965.12it/s]


In [12]:
# 10k craters, 38 mins
csv_path = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\craters_10k.csv"
surviving_craters_df = get_surviving_craters(csv_path)

Processing Craters: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 10039/10039 [03:45<00:00, 44.57it/s]


In [ ]:
# 100k craters, 138 craters in 10 mins (Original speed before any optimization)
csv_path = "C:\\Users\\julia\\OneDrive\\Documents\\Lunar Cratering project David Kipping\\Simulation CSV files\\craters_100k.csv"
surviving_craters_df = get_surviving_craters(csv_path)

Processing Craters:  70%|██████████████████████████████████▏              | 70471/101056 [118:22:31<1:09:37,  7.32it/s]